In [22]:
import pandas as pd 
import json 
import shapefile
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import os 
import json
import numpy as np

In [23]:
def isNested(entry): 
    try: 
        sum(entry)
        return 0 
    except: 
        return 1

In [24]:
def readInJson(fileName):
    try: 
        df = pd.read_json(fileName)
    except: 
        handle = json.load(open(fileName))
        df = pd.DataFrame(handle['features'])
        
    return df

In [25]:
garbageCollectionDate = sorted([i for i in os.listdir() if i.endswith('json')])

In [26]:
lonlatSuburb = pd.read_csv('lonlatSuburb.csv')
vicPostcodes = pd.read_csv('vicPostcodes.csv')

lonlatSuburb['lat'] = lonlatSuburb.lat.apply(lambda x: round(x, 6))
lonlatSuburb['long'] = lonlatSuburb.long.apply(lambda x: round(x, 6))

vicPostcodes['lat'] = vicPostcodes.lat.apply(lambda x: round(x, 6))
vicPostcodes['long'] = vicPostcodes.long.apply(lambda x: round(x, 6))

In [29]:
garbageCollectionDate[24]

'mitchellshire-garbage-collection-zones.geojson'

In [30]:
readInJson(garbageCollectionDate[24]).head(1)

,type,id,geometry,geometry_name,properties
0,Feature,ckan_e39daaff_eb59_455b_b3b8_aaa5f2650903.1,"{'type': 'MultiPolygon', 'coordinates': [[[[14...",geom,"{'rub_day': 'Friday', 'rub_weeks': '1', 'rub_s..."


In [31]:
df = readInJson(garbageCollectionDate[24])

df['geo'] = df.geometry.apply(lambda x: x['coordinates'][0][0])

df = df.explode('geo')

df['geo_nested'] = df.geo.apply(lambda x: isNested(x))
df['lat'] = df.geo.apply(lambda x: round(x[0], 6))
df['long'] = df.geo.apply(lambda x: round(x[1], 6))

df.drop(['geo','id', 'geometry', 'geometry_name',
         'geo_nested', 'type'], axis = 1, inplace = True)

In [35]:
def collectionFrequency(row, wasteType): 
    
    rub_colDay = row[f'{wasteType}_day']
    rub_colFreq = row[f'{wasteType}_weeks']
    
    if rub_colFreq == '2' or rub_colFreq == 2: 
        rub_colFreq = 'Every Second '
    else: 
        rub_colFreq = 'Every '
    
    return rub_colFreq + rub_colDay

df['landfill_colDay'] = df.properties.apply(lambda x: collectionFrequency(x, 'rub'))
df['recyclable_colDay'] = df.properties.apply(lambda x: collectionFrequency(x, 'rec'))

In [36]:
mitchell = lonlatSuburb.merge(df)[['locality', 'long', 'lat', 'landfill_colDay', 'recyclable_colDay',]]\
.sort_values(['locality', 'landfill_colDay', 'recyclable_colDay',])\
        .drop_duplicates(['locality','recyclable_colDay', 'recyclable_colDay',])\
        .sort_values(['locality'])\
.merge(vicPostcodes[['locality', 'postcode']])\
.sort_values(['postcode', 'landfill_colDay', 'recyclable_colDay',])\
.drop_duplicates(subset = ['postcode', 'landfill_colDay', 'recyclable_colDay',])\
.rename(columns = dict(zip(['landfill_colDay', 'recyclable_colDay',], 
                           ['landfill', 'recyclable', 'organics'])))\
.drop(['long', 'lat', 'locality'], axis = 1)\
.melt(id_vars = 'postcode')\
.rename(columns = {'variable':'binInfo_bin_type', 'value':'collection_time'})

mitchell.to_csv('../GBD_mitchell.csv', index = False)